# Project 1

The purpose of this notebook to prepare the DiscGolf and postcode_level_averages data for analysis.

DiscGolf.csv was sourced from https://www.kaggle.com/datasets/lanekatris/pdga-united-states-disc-golf-courses

postcode_level_averages was sourced from https://www.kaggle.com/datasets/hamishgunasekara/average-income-per-zip-code-usa-2018


In [46]:
#Import dependencies
import pandas as pd
import numpy as np
import matplotlib as plt
import requests
import json

In [47]:
#read CSVs and create Pandas DataFrames
discgolf_df = pd.DataFrame(pd.read_csv('Resources/DiscGolf.csv'))
discgolf_clean_df = pd.DataFrame(pd.read_csv('Resources/DiscGolfclean.csv'))
zip_code_df = pd.DataFrame(pd.read_csv('Resources/postcode_level_averages.csv'))

In [48]:
#check discgolfcsv MAJORITY OF COURSES HAVEN NAN RATING BUT DEFINITELY WANT TO KEEP THE DATA, CAN'T USE delete nan to clean up unless it's directed to a single column or all colunms but rating.  
#OR change NaN to 0?
discgolfdf.head(2)

,id,name,city,state,zip,holeCount,rating,latitude,longitude
0,adventist-discovery-park,Adventist DISCovery Park,Opelika,Alabama,36804,3,NaN,32.645412,-85.378280
1,agape-disc-golf-course,Agape Disc Golf Course,Scottsboro,Alabama,35769,9,NaN,34.622819,-86.080692


In [49]:
discgolfdf.dtypes #need to remove where zip is NAN to change zip to int64


id            object
name          object
city          object
state         object
zip           object
holeCount      int64
rating       float64
latitude     float64
longitude    float64
dtype: object

In [50]:
# Disc Golf update zip so dtype is int64 instead of object so we can merge the two dfs

discgolfdf2 = discgolfdf.dropna(subset = ['zip'])
discgolfdf2sort = discgolfdf2.sort_values('zip')
discgolfdf2sort

,id,name,city,state,zip,holeCount,rating,latitude,longitude
2750,school-street-park-disc-golf,School Street Park Disc Golf,Agawam,Massachusetts,1001,9,NaN,42.076168,-72.593475
2742,orchard-hill-disc-golf-umass-amherst,Orchard Hill Disc Golf at UMass Amherst,Amherst,Massachusetts,1002,9,NaN,42.390367,-72.519235
2743,peabody-disc-golf-scouting-woods,Peabody Disc Golf at Scouting Woods,Peabody,Massachusetts,1002,9,4.0,42.526089,-70.950516
2707,barre-falls,Barre Falls,Barre,Massachusetts,1005,18,5.0,42.427868,-72.025355
2764,white-brook-school-disc-golf,White Brook School Disc Golf,Easthampton,Massachusetts,1027,3,NaN,42.253247,-72.672433
...,...,...,...,...,...,...,...,...,...
122,aantiyeik-park,Aant'iyeik Park,Juneau,Alaska,99801,18,NaN,58.382089,-134.736123
146,ravens-bowl,Raven's Bowl,Haines,Alaska,99827,18,NaN,59.234569,-135.443448
132,glacier-point,Glacier Point,Glacier Point,Alaska,99827,18,NaN,59.102500,-135.382222
152,seven-pastures,Seven Pastures,Skagway,Alaska,99840,18,NaN,59.466968,-135.307233


discgolf drop zip NaN  DROP NA DOESN'T WORK BECAUSE IT'S NOT NA, sort doesn't work because it doesn't sort them at the beginning or end.  
DiscGolfClean = manually edited disc golf csv to fix zips, added Modified column (no,yes) and Comment column (none, or modification info)

In [51]:
discgolfcleandf.head()

,id,name,city,state,zip,holeCount,rating,latitude,longitude,Modified,Comments
0,adventist-discovery-park,Adventist DISCovery Park,Opelika,Alabama,36804,3,NaN,32.645412,-85.378280,no,none
1,agape-disc-golf-course,Agape Disc Golf Course,Scottsboro,Alabama,35769,9,NaN,34.622819,-86.080692,no,none
2,aggieland-disc-golf,Aggieland Disc Golf,Hamilton,Alabama,35570,18,NaN,34.142323,-87.988644,no,none
3,agricultural-heritage-park,Agricultural Heritage Park,Auburn,Alabama,36830,9,NaN,32.594459,-85.492334,no,none
4,arab-city-park-844-shoal-creek-trail-35016,Arab City Park,Arab,Alabama,35016,18,NaN,34.317835,-86.481828,no,none


In [52]:
discgolfcleandf.dtypes

id            object
name          object
city          object
state         object
zip            int64
holeCount      int64
rating       float64
latitude     float64
longitude    float64
Modified      object
Comments      object
dtype: object

In [53]:
#check zipcodecsv 
zipcodedf.head(2)

,state,zipcode,total_pop,total_income,country,avg_income
0,AK,0,345220,25354974,USA,73445.843230
1,AK,99501,7690,612242,USA,79615.344603


In [54]:
zipcodedf.dtypes

state            object
zipcode           int64
total_pop         int64
total_income      int64
country          object
avg_income      float64
dtype: object

In [55]:
# Create DraftMerge - inner join DiscGolfcleandf and zipcodedf
#nice reference https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
draft_dgolf_avincome = pd.merge(discgolfcleandf,zipcodedf,how = 'inner',left_on='zip',right_on='zipcode')
draft_dgolf_avincome.head()


,id,name,city,state_x,zip,holeCount,rating,latitude,longitude,Modified,Comments,state_y,zipcode,total_pop,total_income,country,avg_income
0,adventist-discovery-park,Adventist DISCovery Park,Opelika,Alabama,36804,3,NaN,32.645412,-85.378280,no,none,AL,36804,8240,417346,USA,50648.786408
1,agape-disc-golf-course,Agape Disc Golf Course,Scottsboro,Alabama,35769,9,NaN,34.622819,-86.080692,no,none,AL,35769,4170,316625,USA,75929.256595
2,aggieland-disc-golf,Aggieland Disc Golf,Hamilton,Alabama,35570,18,NaN,34.142323,-87.988644,no,none,AL,35570,3930,174684,USA,44448.854962
3,agricultural-heritage-park,Agricultural Heritage Park,Auburn,Alabama,36830,9,NaN,32.594459,-85.492334,no,none,AL,36830,16600,1485585,USA,89493.072289
4,arab-city-park-844-shoal-creek-trail-35016,Arab City Park,Arab,Alabama,35016,18,NaN,34.317835,-86.481828,no,none,AL,35016,7210,385033,USA,53402.635229


In [56]:
#export draft_dgolf_avincome to csv
draft_dgolf_avincome.to_csv('Resources/draft_dgolf_avincome.csv')